# Poisoning of CICIDS-2017 dataset and detecting poisoning using KSSD

Load in processed and poisoned data into this playbook to test KSSD.

## Setup and import required modules

In [ ]:
!pip3 install scikit-learn-intelex

In [ ]:
from sklearnex import patch_sklearn
patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [ ]:
from __future__ import print_function
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split 
from sklearn.metrics import confusion_matrix
from sklearn import model_selection
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestRegressor
from scipy.sparse import csr_matrix, vstack, hstack
from scipy.sparse import coo_matrix
from keras.preprocessing.text import one_hot
from sklearn import metrics
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.semi_supervised import LabelPropagation
from sklearn.semi_supervised import LabelSpreading
#from sklearn.semi_supervised import label_propagation
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.cluster import KMeans
import math
#import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation , Flatten
from sklearn.metrics import log_loss
# from keras.optimizers import SGD
# from keras.layers.normalization import BatchNormalization
# from keras.layers.convolutional import UpSampling2D
# from keras.layers.convolutional import Conv2D, MaxPooling2D, MaxPooling1D
# from keras.layers.embeddings import Embedding
import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import UpSampling2D
from tensorflow.keras.layers import Conv2D, MaxPooling2D, MaxPooling1D
from tensorflow.keras.layers import Embedding
from scipy import sparse
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import PrecisionRecallDisplay

#import random
import sklearn
from sklearn.metrics.pairwise import manhattan_distances
from keras.models import Model
from keras.layers import  Conv1D, multiply, GlobalMaxPool1D, Input , Lambda
import time
import argparse
#import math
from numpy import *
import os.path as osp
import scipy.sparse as sp
import pickle
from sklearn.metrics import accuracy_score

In [ ]:
from sklearn.metrics import precision_recall_curve
def plot_conf_matrix(Y_val, Y_val_preds):
    """
    Plots a nice looking confusion matrix using Seaborn's heatmap()
    """
    sns.set(font_scale=1.5)
    fig, ax = plt.subplots(figsize=(3,3))
    ax = sns.heatmap(confusion_matrix(Y_val, Y_val_preds),
                     annot=True,
                     cbar=False,
                     fmt='d')
    plt.xlabel("Predicted Label")
    plt.ylabel("True Label")

In [ ]:
def plot_conf_matrix_from_matrix(cm, title):
    """
    Plots a nice looking confusion matrix using Seaborn's heatmap()
    """
    sns.set(font_scale=1.5)
    fig, ax = plt.subplots(figsize=(3,3))
    ax = sns.heatmap(cm,
                     annot=True,
                     cbar=False,
                     fmt='d')
    plt.title(label=title)
    plt.xlabel("Predicted Label")
    plt.ylabel("True Label")

Load Data

In [ ]:
import pandas as pd
# input_data = pd.read_csv('/content/drive/MyDrive/CICIDS/converted_processed_1_percent_sample.csv', index_col='Unnamed: 0')
# input_data = pd.read_csv('/content/drive/MyDrive/CICIDS/converted_processed_5_percent_sample.csv', index_col='Unnamed: 0')
input_data = pd.read_csv('/content/drive/MyDrive/CICIDS/Converted-Processed-CICIDS.csv', index_col='Unnamed: 0')

input_tables = input_data.copy()
input_tables.reset_index(inplace=True)
input_tables.drop(['index'], axis=1, inplace=True)

In [ ]:
dl2 = input_tables.copy(deep=True)
print("Pre-conversion values:\n", dl2['Label'].value_counts())
for index, row in dl2.iterrows():
  if row['Label'] == 'BENIGN':
    dl2.at[index, 'Label'] = 0
  else:
    dl2.at[index, 'Label'] = 1
print("")
print("Converted values:\n", dl2['Label'].value_counts())
dl2 = dl2['Label']
dl2 = dl2.astype('int')

## Main Program Start

In [ ]:
### /content/drive/MyDrive/Dissertation/KNN
## Loading
x_train_filename = '/content/drive/MyDrive/Dissertation/KNN/X_train.sav'
X_train = pickle.load(open(x_train_filename, 'rb'))
x_test_filename = '/content/drive/MyDrive/Dissertation/KNN/X_test.sav'
X_test = pickle.load(open(x_test_filename, 'rb'))
x_val_filename = '/content/drive/MyDrive/Dissertation/KNN/X_val.sav'
X_val = pickle.load(open(x_val_filename, 'rb'))
y_train_filename = '/content/drive/MyDrive/Dissertation/KNN/Y_train.sav'
Y_train = pickle.load(open(y_train_filename, 'rb'))
y_test_filename = '/content/drive/MyDrive/Dissertation/KNN/Y_test.sav'
Y_test = pickle.load(open(y_test_filename, 'rb'))
y_val_filename = '/content/drive/MyDrive/Dissertation/KNN/Y_val.sav'
Y_val = pickle.load(open(y_val_filename, 'rb'))

Model Definition

In [ ]:
X_train_NoAttack = X_train.copy()
Y_train_NoAttack = Y_train.copy()

X_val_NoAttack = X_val.copy()
Y_val_NoAttack = Y_val.copy()

row_train_NoAttack, column_train_NoAttack = X_train_NoAttack.shape
model_main = Sequential()
model_main.add(Conv1D(16,2, strides=2, padding='same', input_shape=(69, 1)))
model_main.add(MaxPooling1D(pool_size=(4), strides=(2)))
model_main.add(Conv1D(32,2, strides=2, padding='same', input_shape=(69, 1)))
model_main.add(MaxPooling1D(pool_size=(4), strides=(2)))
model_main.add(Conv1D(64,2, strides=2, padding='same', input_shape=(69, 1)))
model_main.add(Flatten())
model_main.add(Dense(1, activation='sigmoid'))
model_main.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model_main.fit(X_train_NoAttack, Y_train_NoAttack, epochs=200, verbose=1)

In [ ]:
Y_CNN_NoAttack = model_main.predict(X_test, verbose=0)
Y_predict_NoAttack = [0]*len(Y_CNN_NoAttack)

for i in range(len(Y_CNN_NoAttack)):
    if Y_CNN_NoAttack[i] < 0.5:
            Y_CNN_NoAttack[i] = 0
    else:
            Y_CNN_NoAttack[i] = 1

for i in range(len(Y_CNN_NoAttack)):
    Y_predict_NoAttack[i] = int(Y_CNN_NoAttack[i])

In [ ]:
### Saving
model_main_filename = '/content/drive/MyDrive/Dissertation/KNN/model_main.sav'
pickle.dump(model_main, open(model_main_filename, 'wb'))
X_train_NoAttack_filename = '/content/drive/MyDrive/Dissertation/KNN/X_train_NoAttack.sav'
pickle.dump(X_train_NoAttack, open(X_train_NoAttack_filename, 'wb'))
Y_train_NoAttack_filename = '/content/drive/MyDrive/Dissertation/KNN/Y_train_NoAttack.sav'
pickle.dump(Y_train_NoAttack, open(Y_train_NoAttack_filename, 'wb'))
X_val_NoAttack_filename = '/content/drive/MyDrive/Dissertation/KNN/X_val_NoAttack.sav'
pickle.dump(X_val_NoAttack, open(X_val_NoAttack_filename, 'wb'))
Y_val_NoAttack_filename = '/content/drive/MyDrive/Dissertation/KNN/Y_val_NoAttack.sav'
pickle.dump(Y_val_NoAttack, open(Y_val_NoAttack_filename, 'wb'))
Y_CNN_NoAttack_filename = '/content/drive/MyDrive/Dissertation/KNN/Y_CNN_NoAttack.sav'
pickle.dump(Y_CNN_NoAttack, open(Y_CNN_NoAttack_filename, 'wb'))
Y_predict_NoAttack_filename = '/content/drive/MyDrive/Dissertation/KNN/Y_predict_NoAttack.sav'
pickle.dump(Y_predict_NoAttack, open(Y_predict_NoAttack_filename, 'wb'))

In [ ]:
# ### Loading
# model_main_filename = '/content/drive/MyDrive/Dissertation/KNN/model_main.sav'
# model_main = pickle.load(open(model_main_filename, 'rb'))
# X_train_NoAttack_filename = '/content/drive/MyDrive/Dissertation/KNN/X_train_NoAttack.sav'
# X_train_NoAttack = pickle.load(open(X_train_NoAttack_filename, 'rb'))
# Y_train_NoAttack_filename = '/content/drive/MyDrive/Dissertation/KNN/Y_train_NoAttack.sav'
# Y_train_NoAttack = pickle.load(open(Y_train_NoAttack_filename, 'rb'))
# X_val_NoAttack_filename = '/content/drive/MyDrive/Dissertation/KNN/X_val_NoAttack.sav'
# X_val_NoAttack = pickle.load(open(X_val_NoAttack_filename, 'rb'))
# Y_val_NoAttack_filename = '/content/drive/MyDrive/Dissertation/KNN/Y_val_NoAttack.sav'
# Y_val_NoAttack = pickle.load(open(Y_val_NoAttack_filename, 'rb'))
# Y_CNN_NoAttack_filename = '/content/drive/MyDrive/Dissertation/KNN/Y_CNN_NoAttack.sav'
# Y_CNN_NoAttack = pickle.load(open(Y_CNN_NoAttack_filename, 'rb'))
# Y_predict_NoAttack_filename = '/content/drive/MyDrive/Dissertation/KNN/Y_predict_NoAttack.sav'
# Y_predict_NoAttack = pickle.load(open(Y_predict_NoAttack_filename, 'rb'))

Result of Model without attack on X_test

In [ ]:
print("********************************Result of Model without attack******************************************************************")
# loss, accuracy = model_main.evaluate(X_train_NoAttack, Y_train_NoAttack, verbose=2)
#loss, accuracy = model_main.evaluate(X_train_NoAttack, Y_train_NoAttack, verbose=2)
loss, accuracy = model_main.evaluate(X_train_NoAttack, Y_train_NoAttack, verbose=2)

print('Accuracy for Train set: %f' % (accuracy*100))
print('Loss for Train set: %f' % (loss))
print("                                                                   ")

#loss, accuracy = model_main.evaluate(X_val_NoAttack, Y_val_NoAttack, verbose=2)
loss, accuracy = model_main.evaluate(X_val_NoAttack, Y_val_NoAttack, verbose=2)
print('Accuracy for Validation set: %f' % (accuracy*100))
print('Loss for Train Validation set: %f' % (loss))
print("                                                                   ")

loss, accuracy = model_main.evaluate(X_test, Y_test, verbose=2)
print('Accuracy for Test set: %f' % (accuracy*100))
print('Loss for Test set:: %f' % (loss))
print("                                                                   ")

con_matrix = confusion_matrix(Y_test, Y_predict_NoAttack)
print(con_matrix, "\n")

TN_NoAttack, FP_NoAttack, FN_NoAttack, TP_NoAttack = confusion_matrix((Y_test.astype(int).tolist()), Y_predict_NoAttack).ravel()
print("TN_NoAttack:",TN_NoAttack)
print("FP_NoAttack:",FP_NoAttack)
print("FN_NoAttack:",FN_NoAttack)
print("TP_NoAttack:",TP_NoAttack)
print("                                                                   ")

if (FP_NoAttack + TN_NoAttack) > 0:
        FPR_NoAttack = FP_NoAttack/(FP_NoAttack + TN_NoAttack)
        print("The FPR_NoAttack result:", FPR_NoAttack)
            
if (FP_NoAttack + TN_NoAttack) > 0:
        TPR_NoAttack=TP_NoAttack/(TP_NoAttack+FN_NoAttack)
        print("The TPR_NoAttack result=", TPR_NoAttack)
            
if (TN_NoAttack + FP_NoAttack) > 0:
    TNR_NoAttack = TN_NoAttack/(TN_NoAttack + FP_NoAttack)
    print("The TNR_NoAttack result:", TNR_NoAttack)
            
if (FN_NoAttack + TP_NoAttack) > 0:
    FNR_NoAttack = FN_NoAttack/(FN_NoAttack + TP_NoAttack)
    print("The FNR_NoAttack result:", FNR_NoAttack)
            
if ((TN_NoAttack/(TN_NoAttack + FP_NoAttack)) + (TP_NoAttack/(TP_NoAttack + FP_NoAttack))) > 0:
    AUC_NoAttack = 1/(2*((TN_NoAttack/(TN_NoAttack + FP_NoAttack)) + (TP_NoAttack/(TP_NoAttack + FP_NoAttack))))
    print("The AUC_NoAttack result:", AUC_NoAttack)
            
if  (TP_NoAttack + TN_NoAttack + FP_NoAttack + FN_NoAttack) > 0:
    ACC_NoAttack = (TP_NoAttack + TN_NoAttack)/(TP_NoAttack + TN_NoAttack + FP_NoAttack + FN_NoAttack)
    print("The ACC_NoAttack result:", ACC_NoAttack)
            
if  ((TP_NoAttack + FP_NoAttack) * (TP_NoAttack + FN_NoAttack) * (TN_NoAttack + FP_NoAttack) * (TN_NoAttack + FN_NoAttack)) > 0:
    MCC_NoAttack = (TP_NoAttack * TN_NoAttack - FP_NoAttack * FN_NoAttack)/math.sqrt((TP_NoAttack + FP_NoAttack) * (TP_NoAttack + FN_NoAttack) * (TN_NoAttack + FP_NoAttack) * (TN_NoAttack + FN_NoAttack))
    print("The Matthews correlation coefficient result:", MCC_NoAttack)
print("                                                                                                                               ")
print("*****************************************************End of Without Attack part************************************************")

********************************Result of Model without attack******************************************************************
53077/53077 - 87s - loss: 0.0226 - acc: 0.9931 - 87s/epoch - 2ms/step
Accuracy for Train set: 99.313551
Loss for Train set: 0.022632
                                                                   
17693/17693 - 29s - loss: 0.0231 - acc: 0.9928 - 29s/epoch - 2ms/step
Accuracy for Validation set: 99.283755
Loss for Train Validation set: 0.023121
                                                                   
17693/17693 - 29s - loss: 0.0260 - acc: 0.9929 - 29s/epoch - 2ms/step
Accuracy for Test set: 99.287289
Loss for Test set:: 0.025963
                                                                   
[[452810   1654]
 [  2381 109304]] 

TN_NoAttack: 452810
FP_NoAttack: 1654
FN_NoAttack: 2381
TP_NoAttack: 109304
                                                                   
The FPR_NoAttack result: 0.003639452189832418
The TPR_NoAttack result= 0

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:55: RuntimeWarning: overflow encountered in long_scalars


## Label Flipping Attack

In [ ]:
print("*****************************************************Label Flipping Attack*****************************************************")
print("                                                                                                                               ")
#************************** 
# Finding Malware of Train data
malware_train = X_train.copy(deep=True)
y_malware_train = Y_train.copy(deep=True)
print("malware_train:", malware_train.shape)
cl_malware = list()
z_m = 0
count_m = 0
drop_list = []
for i, j in enumerate(Y_train):
    if j == 0:
        drop_list.append(X_train.index[i])
        count_m = count_m + 1
    else:
        cl_malware.insert(z_m, 1)
        z_m = z_m + 1 
drop_list.sort(reverse=True)
malware_train.drop(drop_list, inplace=True)
y_malware_train.drop(drop_list, inplace=True)

#***************************
#Finding Benign of Train data
cl_X_train = list(Y_train) 
benign_train = X_train.copy(deep=True)
y_benign_train = Y_train.copy(deep=True)
print("benign_train:", benign_train.shape)
z_b = 0
count_b = 0
cl_benign = list()
drop_list = []
for i, j in enumerate(cl_X_train):
    if j == 1:
        drop_list.append(X_train.index[i])
        count_b = count_b+1
    else:
        cl_benign.insert(z_b, 1)
        z_b = z_b+1
drop_list.sort(reverse=True)
benign_train.drop(drop_list, inplace=True)
y_benign_train.drop(drop_list, inplace=True)

print("***********Size of Each Data Part:**********")        
print("malware_train:", malware_train.shape)
print("benign_train:", benign_train.shape)
#***************************************************
row_malware_train, column_malware_train = malware_train.shape

X_train_LFA = X_train.copy(deep=True)
Y_train_LFA = Y_train.copy(deep=True)

row_train_LFA, column_train_LFA = X_train_LFA.shape

In [ ]:
### Saving
malware_train_filename = '/content/drive/MyDrive/Dissertation/KNN/malware_train.sav'
pickle.dump(malware_train, open(malware_train_filename, 'wb'))
y_malware_train_filename = '/content/drive/MyDrive/Dissertation/KNN/y_malware_train.sav'
pickle.dump(y_malware_train, open(y_malware_train_filename, 'wb'))
benign_train_filename = '/content/drive/MyDrive/Dissertation/KNN/benign_train.sav'
pickle.dump(benign_train, open(benign_train_filename, 'wb'))
y_benign_train_filename = '/content/drive/MyDrive/Dissertation/KNN/y_benign_train.sav'
pickle.dump(y_benign_train, open(y_benign_train_filename, 'wb'))
X_train_LFA_filename = '/content/drive/MyDrive/Dissertation/KNN/X_train_LFA.sav'
pickle.dump(X_train_LFA, open(X_train_LFA_filename, 'wb'))
Y_train_LFA_filename = '/content/drive/MyDrive/Dissertation/KNN/Y_train_LFA.sav'
pickle.dump(Y_train_LFA, open(Y_train_LFA_filename, 'wb'))

In [ ]:
# ### loading
# malware_train_filename = '/content/drive/MyDrive/Dissertation/KNN/malware_train.sav'
# malware_train = pickle.load(open(malware_train_filename, 'rb'))
# y_malware_train_filename = '/content/drive/MyDrive/Dissertation/KNN/y_malware_train.sav'
# y_malware_train = pickle.load(open(y_malware_train_filename, 'rb'))
# benign_train_filename = '/content/drive/MyDrive/Dissertation/KNN/benign_train.sav'
# benign_train = pickle.load(open(benign_train_filename, 'rb'))
# y_benign_train_filename = '/content/drive/MyDrive/Dissertation/KNN/y_benign_train.sav'
# y_benign_train = pickle.load(open(y_benign_train_filename, 'rb'))
# X_train_LFA_filename = '/content/drive/MyDrive/Dissertation/KNN/X_train_LFA.sav'
# X_train_LFA = pickle.load(open(X_train_LFA_filename, 'rb'))
# Y_train_LFA_filename = '/content/drive/MyDrive/Dissertation/KNN/Y_train_LFA.sav'
# Y_train_LFA = pickle.load(open(Y_train_LFA_filename, 'rb'))

# row_train_LFA, column_train_LFA = X_train_LFA.shape

In [ ]:
clusterer = KMeans(n_clusters=2, random_state=10)
#X=X_train_LFA.toarray()
X = X_train_LFA.copy(deep=True)
t0 = time.time()
cluster_labels = clusterer.fit_predict(X)
sample_silhouette_values = silhouette_samples(X, cluster_labels)
#print("sample_silhouette_values = ",sample_silhouette_values)
t1 = time.time()
print("Time for creating silhouette samples: ",t1-t0)

In [ ]:
### Saving silhouette_samples
silhouette_samples_filename = '/content/drive/MyDrive/Dissertation/KNN/silhouette_samples.pkl'
pickle.dump(sample_silhouette_values, open(silhouette_samples_filename, 'wb'))
clusterer_filename = '/content/drive/MyDrive/Dissertation/KNN/clusterer.pkl'
pickle.dump(clusterer, open(clusterer_filename, 'wb'))
cluster_labels_filename = '/content/drive/MyDrive/Dissertation/KNN/cluster_labels.pkl'
pickle.dump(cluster_labels, open(cluster_labels_filename, 'wb'))

In [ ]:
## Loading silhouette_samples
silhouette_samples_filename = '/content/drive/MyDrive/Dissertation/KNN/silhouette_samples.pkl'
sample_silhouette_values = pickle.load(open(silhouette_samples_filename, 'rb'))
clusterer_filename = '/content/drive/MyDrive/Dissertation/KNN/clusterer.pkl'
clusterer = pickle.load(open(clusterer_filename, 'rb'))
cluster_labels_filename = '/content/drive/MyDrive/Dissertation/KNN/cluster_labels.pkl'
cluster_labels = pickle.load(open(cluster_labels_filename, 'rb'))

In [ ]:
t0 = time.time()
print("Values:\n", Y_train_LFA.value_counts())
flipped_Y_train = list(Y_train_LFA)
# flipped_Y_train = Y_train_LFA.copy(deep=True)
counter = 0

### For CICIDS 2017 - replace silhouette_level with these values for different levels of poisoning
### 0.05 = Original label flipping level
### 0.15 = 5% label flipping = -0.04908  = 5585
### 0.29 = 10% label flipping = 0.0277734 = 11168
### 0.335 = 15% label flipping =  0.066497  = 16752
### 0.36 = 20% label flipping = 0.0932973  = 22337
### 0.405 = 25% label flipping = 0.115875  = 27921

silhouette_level = 0.405
for new_index in range(row_train_LFA): 
    if (sample_silhouette_values[new_index]<silhouette_level):
        flipped_Y_train[new_index] = abs(flipped_Y_train[new_index]-1)     # flipped_Y_train[new_index]=1
        counter = counter + 1

print("Flipped counter:", counter)
p1 = 1363891 + 334554
print(str((counter/p1)*100)+ "%")
print("0:", str(flipped_Y_train.count(0)))
print("1:", str(flipped_Y_train.count(1)))
t1 = time.time()
print("Time for Label Flipping Attack: ",t1-t0)
print("                                                         ") 

Values:
 0    1363891
1     334554
dtype: int64
Flipped counter: 428497
25.228782798383225%
0: 1223594
1: 474851
Time for Label Flipping Attack:  0.6700105667114258
                                                         


In [ ]:
### Saving
flipped_Y_train_filename = '/content/drive/MyDrive/Dissertation/KNN/5/flipped_Y_train.pkl'
pickle.dump(flipped_Y_train, open(flipped_Y_train_filename, 'wb'))

In [ ]:
# ## Loading
# flipped_Y_train_filename = '/content/drive/MyDrive/Dissertation/KNN/5/flipped_Y_train.pkl'
# flipped_Y_train = pickle.load(open(flipped_Y_train_filename, 'rb'))

In [ ]:
#**************************************************************************
model_main_LFA_Final = Sequential()
model_main_LFA_Final.add(Conv1D(16,2, strides=2, padding='same', input_shape=(69,1)))
model_main_LFA_Final.add(MaxPooling1D(pool_size=(4), strides=(2)))
model_main_LFA_Final.add(Conv1D(32,2, strides=2, padding='same', input_shape=(69,1)))
model_main_LFA_Final.add(MaxPooling1D(pool_size=(4), strides=(2)))
model_main_LFA_Final.add(Conv1D(64,2, strides=2, padding='same', input_shape=(69,1)))
model_main_LFA_Final.add(Flatten())
model_main_LFA_Final.add(Dense(1, activation='sigmoid'))
model_main_LFA_Final.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model_main_LFA_Final.fit(X_train_LFA, np.array(flipped_Y_train), epochs=200, verbose=1)

In [ ]:
Y_predict_LFA=model_main_LFA_Final.predict(X_test, verbose=0)
Y_predict_LFA_Final=[0]*len(Y_predict_LFA)

for i in range(len(Y_predict_LFA)):
    if Y_predict_LFA[i]<0.5:
            Y_predict_LFA[i] = 0
    else:
            Y_predict_LFA[i] = 1

for i in range(len(Y_predict_LFA)):
    Y_predict_LFA_Final[i] = int(Y_predict_LFA[i])  

In [ ]:
### Saving
model_main_LFA_Final_filename = '/content/drive/MyDrive/Dissertation/KNN/5/model_main_LFA_Final.pkl'
pickle.dump(model_main_LFA_Final, open(model_main_LFA_Final_filename, 'wb'))
Y_predict_LFA_filename = '/content/drive/MyDrive/Dissertation/KNN/5/Y_predict_LFA.pkl'
pickle.dump(Y_predict_LFA, open(Y_predict_LFA_filename, 'wb'))
Y_predict_LFA_Final_filename = '/content/drive/MyDrive/Dissertation/KNN/5/Y_predict_LFA_Final.pkl'
pickle.dump(Y_predict_LFA_Final, open(Y_predict_LFA_Final_filename, 'wb'))

In [ ]:
# ### Loading
# model_main_LFA_Final_filename = '/content/drive/MyDrive/Dissertation/KNN/5/model_main_LFA_Final.pkl'
# model_main_LFA_Final = pickle.load(open(model_main_LFA_Final_filename, 'rb'))
# Y_predict_LFA_filename = '/content/drive/MyDrive/Dissertation/KNN/5/Y_predict_LFA.pkl'
# Y_predict_LFA = pickle.load(open(Y_predict_LFA_filename, 'rb'))
# Y_predict_LFA_Final_filename = '/content/drive/MyDrive/Dissertation/KNN/5/Y_predict_LFA_Final.pkl'
# Y_predict_LFA_Final = pickle.load(open(Y_predict_LFA_Final_filename, 'rb'))

Result of Model with LFA

In [ ]:
print("********************************Result of Model with LFA attack **************************************************************")
print("                                                                                                                              ")
loss, accuracy = model_main_LFA_Final.evaluate(X_train_LFA, np.array(flipped_Y_train), verbose=2)
print('Accuracy for Train set: %f' % (accuracy*100))
print('Loss for Train set: %f' % (loss))
print("                                                                   ")

loss, accuracy = model_main_LFA_Final.evaluate(X_test, Y_test, verbose=2)
print('Accuracy for Test set: %f' % (accuracy*100))
print('Loss for Test set:: %f' % (loss))
print("                                                                   ")

TN_LFA, FP_LFA, FN_LFA, TP_LFA = confusion_matrix(Y_test,  Y_predict_LFA_Final).ravel()
print("TN_LFA:", TN_LFA)
print("FP_LFA:", FP_LFA)
print("FN_LFA:", FN_LFA)
print("TP_LFA:", TP_LFA)
print("                                                                   ")

if (FP_LFA+TN_LFA) > 0:
        FPR_LFA=FP_LFA/(FP_LFA+TN_LFA)
        print("The FPR_LFA result=", FPR_LFA)
            
if (FP_LFA+TN_LFA) > 0:
        TPR_LFA = TP_LFA/(TP_LFA+FN_LFA)
        print("The TPR_LFA result:", TPR_LFA)
            
if (TN_LFA+FP_LFA) > 0:
    TNR_LFA = TN_LFA/(TN_LFA+FP_LFA)
    print("The TNR_LFA result:", TNR_LFA)
            
if (FN_LFA+TP_LFA) > 0:
    FNR_LFA = FN_LFA/(FN_LFA+TP_LFA)
    print("The FNR_LFA result:", FNR_LFA)
            
if ((TN_LFA/(TN_LFA+FP_LFA))+(TP_LFA/(TP_LFA+FP_LFA))) > 0:
    AUC_LFA = 1/(2*((TN_LFA/(TN_LFA+FP_LFA))+(TP_LFA/(TP_LFA+FP_LFA))))
    print("The AUC_LFA result:", AUC_LFA)
            
if  (TP_LFA+TN_LFA+FP_LFA+FN_LFA) > 0:
    ACC_LFA = (TP_LFA+TN_LFA)/(TP_LFA+TN_LFA+FP_LFA+FN_LFA)
    print("The ACC_LFA result:", ACC_LFA)
            
if  ((TP_LFA+FP_LFA)*(TP_LFA+FN_LFA)*(TN_LFA+FP_LFA)*(TN_LFA+FN_LFA)) > 0:
    MCC_LFA = (TP_LFA*TN_LFA-FP_LFA*FN_LFA)/math.sqrt((TP_LFA+FP_LFA)*(TP_LFA+FN_LFA)*(TN_LFA+FP_LFA)*(TN_LFA+FN_LFA))
    print("The Matthews correlation coefficient result:", MCC_LFA)
print("                                                                                                                               ")
print("************************************************End of Label Flipping Attack part**********************************************")

********************************Result of Model with LFA attack **************************************************************
                                                                                                                              
53077/53077 - 86s - loss: 0.0547 - acc: 0.9814 - 86s/epoch - 2ms/step
Accuracy for Train set: 98.141950
Loss for Train set: 0.054745
                                                                   
17693/17693 - 36s - loss: 15.0193 - acc: 0.7358 - 36s/epoch - 2ms/step
Accuracy for Test set: 73.580807
Loss for Test set:: 15.019323
                                                                   
TN_LFA: 358078
FP_LFA: 96386
FN_LFA: 53186
TP_LFA: 58499
                                                                   
The FPR_LFA result= 0.2120872060273201
The TPR_LFA result: 0.5237856471325604
The TNR_LFA result: 0.7879127939726799
The FNR_LFA result: 0.4762143528674397
The AUC_LFA result: 0.4289614448034144
The ACC_LFA result: 0.7

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: RuntimeWarning: overflow encountered in long_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45: RuntimeWarning: overflow encountered in long_scalars


KNN Based Semi-Supervised Defense(KSD)

In [ ]:
print("*****************************************************KNN Based Semi-Supervised Defense(KSD)************************************")
print("                                                                                                                               ")

X_train_KNN = X_train.copy(deep=True)
Y_train_KNN = flipped_Y_train[:]
# Y_train_KNN = flipped_Y_train.copy(deep=True)

X_val_KNN = X_val.copy(deep=True)
# Y_val_KNN = Y_val[:]
Y_val_KNN = Y_val.copy(deep=True)

row_train_KNN, column_train_KNN = X_train_KNN.shape
    
Number_of_flipped_label = int(row_train_KNN/50)
Y_train_corrected_By_KNN = list(Y_train_KNN)

c = 0
m = 0

*****************************************************KNN Based Semi-Supervised Defense(KSD)************************************
                                                                                                                               


In [ ]:
### Saving
X_train_KNN_filename = '/content/drive/MyDrive/Dissertation/KSSD/5/X_train_KNN.pkl'
pickle.dump(X_train_KNN, open(X_train_KNN_filename, 'wb'))
Y_train_KNN_filename = '/content/drive/MyDrive/Dissertation/KSSD/5/Y_train_KNN.pkl'
pickle.dump(Y_train_KNN, open(Y_train_KNN_filename, 'wb'))
X_val_KNN_filename = '/content/drive/MyDrive/Dissertation/KSSD/5/X_val_KNN.pkl'
pickle.dump(X_val_KNN, open(X_val_KNN_filename, 'wb'))
Y_val_KNN_filename = '/content/drive/MyDrive/Dissertation/KSSD/5/Y_val_KNN.pkl'
pickle.dump(Y_val_KNN, open(Y_val_KNN_filename, 'wb'))
Number_of_flipped_label_filename = '/content/drive/MyDrive/Dissertation/KSSD/5/Number_of_flipped_label.pkl'
pickle.dump(Number_of_flipped_label, open(Number_of_flipped_label_filename, 'wb'))
Y_train_corrected_By_KNN_filename = '/content/drive/MyDrive/Dissertation/KSSD/5/Y_train_corrected_By_KNN.pkl'
pickle.dump(Y_train_corrected_By_KNN, open(Y_train_corrected_By_KNN_filename, 'wb'))

In [ ]:
# ### Loading
# X_train_KNN_filename = '/content/drive/MyDrive/Dissertation/KSSD/5/X_train_KNN.pkl'
# X_train_KNN = pickle.load(open(X_train_KNN_filename, 'rb'))
# Y_train_KNN_filename = '/content/drive/MyDrive/Dissertation/KSSD/5/Y_train_KNN.pkl'
# Y_train_KNN = pickle.load(open(Y_train_KNN_filename, 'rb'))
# X_val_KNN_filename = '/content/drive/MyDrive/Dissertation/KSSD/5/X_val_KNN.pkl'
# X_val_KNN = pickle.load(open(X_val_KNN_filename, 'rb'))
# Y_val_KNN_filename = '/content/drive/MyDrive/Dissertation/KSSD/5/Y_val_KNN.pkl'
# Y_val_KNN = pickle.load(open(Y_val_KNN_filename, 'rb'))
# Number_of_flipped_label_filename = '/content/drive/MyDrive/Dissertation/KSSD/5/Number_of_flipped_label.pkl'
# Number_of_flipped_label = pickle.load(open(Number_of_flipped_label_filename, 'rb'))
# Y_train_corrected_By_KNN_filename = '/content/drive/MyDrive/Dissertation/KSSD/5/Y_train_corrected_By_KNN.pkl'
# Y_train_corrected_By_KNN = pickle.load(open(Y_train_corrected_By_KNN_filename, 'rb'))

In [ ]:
t2 = time.time()

for i in list(range(Number_of_flipped_label)):
        row_KNN = X_train_KNN.head(i+1).tail(1)

        distances = sklearn.metrics.pairwise.manhattan_distances(row_KNN, X_val_KNN)
        indices = distances.argsort()[:10]
        d=indices[0]
        a=d[0:10]
    
        F=0
        for j in range(len(a)):
                t = a[j]
                F = F+Y_val_KNN.iloc[t]
        fraction=F/10
        if fraction >= 0.5:
            # Y_train_corrected_By_KNN[i] = 1
            Y_train_corrected_By_KNN[i] = 0
            m = m + 1
        else: 
            # Y_train_corrected_By_KNN[i] = 0
            Y_train_corrected_By_KNN[i] = 1
            c = c + 1
Y_train_corrected_By_KNN_Final=np.array(Y_train_corrected_By_KNN)  
t3=time.time()
print("Time for KNN Based Semi-Supervised Defense(KSD) =",t3-t2)
print("                                                                   ")

Time for KNN Based Semi-Supervised Defense(KSD) = 8801.104440689087
                                                                   


In [ ]:
### Saving
Y_train_corrected_By_KNN_Final_filename = '/content/drive/MyDrive/Dissertation/KSSD/5/Y_train_corrected_By_KNN_Final.pkl'
pickle.dump(Y_train_corrected_By_KNN_Final, open(Y_train_corrected_By_KNN_Final_filename, 'wb'))

In [ ]:
# ### Loading
# Y_train_corrected_By_KNN_Final_filename = '/content/drive/MyDrive/Dissertation/KSSD/5/Y_train_corrected_By_KNN_Final.pkl'
# Y_train_corrected_By_KNN_Final = pickle.load(open(Y_train_corrected_By_KNN_Final_filename, 'rb'))

In [ ]:
model_main_KNN = Sequential()
model_main_KNN.add(Conv1D(16,2, strides=2, padding='same', input_shape=(69,1)))
model_main_KNN.add(MaxPooling1D(pool_size=(4), strides=(2)))
model_main_KNN.add(Conv1D(32,2, strides=2, padding='same', input_shape=(69,1)))
model_main_KNN.add(MaxPooling1D(pool_size=(4), strides=(2)))
model_main_KNN.add(Conv1D(64,2, strides=2, padding='same', input_shape=(69,1)))
model_main_KNN.add(Flatten())
model_main_KNN.add(Dense(1, activation='sigmoid'))
model_main_KNN.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model_main_KNN.fit(X_train_KNN,Y_train_corrected_By_KNN_Final, epochs=20, batch_size=32, verbose=1)

Epoch 1/20
53077/53077 [==============================] - 159s 3ms/step - loss: 0.2118 - acc: 0.9207
Epoch 2/20
53077/53077 [==============================] - 158s 3ms/step - loss: 0.1803 - acc: 0.9368
Epoch 3/20
53077/53077 [==============================] - 158s 3ms/step - loss: 0.1743 - acc: 0.9398
Epoch 4/20
53077/53077 [==============================] - 157s 3ms/step - loss: 0.1709 - acc: 0.9413
Epoch 5/20
53077/53077 [==============================] - 158s 3ms/step - loss: 0.1686 - acc: 0.9423
Epoch 6/20
53077/53077 [==============================] - 158s 3ms/step - loss: 0.1668 - acc: 0.9432
Epoch 7/20
53077/53077 [==============================] - 159s 3ms/step - loss: 0.1650 - acc: 0.9439
Epoch 8/20
53077/53077 [==============================] - 158s 3ms/step - loss: 0.1637 - acc: 0.9446
Epoch 9/20
53077/53077 [==============================] - 159s 3ms/step - loss: 0.1627 - acc: 0.9453
Epoch 10/20
53077/53077 [==============================] - 158s 3ms/step - loss: 0.1617 - a

In [ ]:
Y_predict_KNN = model_main_KNN.predict(X_test, verbose=0)
Y_predict_KNN_Final = [0]*len(Y_predict_KNN)
for i in range(len(Y_predict_KNN)):
    if Y_predict_KNN[i] < 0.5:
            Y_predict_KNN[i] = 0
    else:
            Y_predict_KNN[i] = 1

for i in range(len(Y_predict_KNN)):
    Y_predict_KNN_Final[i] = int(Y_predict_KNN[i])

In [ ]:
### Saving
model_main_KNN_filename = '/content/drive/MyDrive/Dissertation/KSSD/5/model_main_KNN.pkl'
pickle.dump(model_main_KNN, open(model_main_KNN_filename, 'wb'))
Y_predict_KNN_filename = '/content/drive/MyDrive/Dissertation/KSSD/5/Y_predict_KNN.pkl'
pickle.dump(Y_predict_KNN, open(Y_predict_KNN_filename, 'wb'))
Y_predict_KNN_Final_filename = '/content/drive/MyDrive/Dissertation/KSSD/5/Y_predict_KNN_Final.pkl'
pickle.dump(Y_predict_KNN_Final, open(Y_predict_KNN_Final_filename, 'wb'))

In [ ]:
# ### Loading
# model_main_KNN_filename = '/content/drive/MyDrive/Dissertation/KSSD/5/model_main_KNN.pkl'
# model_main_KNN = pickle.load(open(model_main_KNN_filename, 'rb'))
# Y_predict_KNN_filename = '/content/drive/MyDrive/Dissertation/KSSD/5/Y_predict_KNN.pkl'
# Y_predict_KNN = pickle.load(open(Y_predict_KNN_filename, 'rb'))
# Y_predict_KNN_Final_filename = '/content/drive/MyDrive/Dissertation/KSSD/5/Y_predict_KNN_Final.pkl'
# Y_predict_KNN_Final = pickle.load(open(Y_predict_KNN_Final_filename, 'rb'))

Result of Model After KNN Based Defense

In [ ]:
print("************************Result After KNN_Based Defense************************************************************************")
print("                                                                                                                               ")

loss, accuracy = model_main_KNN.evaluate(X_train_KNN, np.asarray(Y_train_KNN), verbose=0)
print('Accuracy for Train set: %f' % (accuracy*100))
print('Loss for Train set: %f' % (loss))
print("                                                                   ")

loss, accuracy = model_main_KNN.evaluate(X_test, Y_test, batch_size=32, verbose=0)
print('Accuracy After KNN-Based Defense: %f' % (accuracy*100))
print('Loss After KNN-Based Defense: %f' % (loss))
print("                                                                   ")

TN_KNN, FP_KNN, FN_KNN, TP_KNN = confusion_matrix(Y_test, Y_predict_KNN_Final).ravel()
print("TN_KNN:",TN_KNN)
print("FP_KNN:",FP_KNN)
print("FN_KNN:",FN_KNN)
print("TP_KNN:",TP_KNN)
print("                                                                   ")

if (FP_KNN+TN_KNN) > 0:
        FPR_KNN=FP_KNN/(FP_KNN+TN_KNN)
        print("The FPR_KNN result:", FPR_KNN)
            
if (FP_KNN+TN_KNN) > 0:
        TPR_KNN=TP_KNN/(TP_KNN+FN_KNN)
        print("The TPR_KNN result:", TPR_KNN)
            
if (TN_KNN+FP_KNN) > 0:
    TNR_KNN=TN_KNN/(TN_KNN+FP_KNN)
    print("The TNR_KNN result:", TNR_KNN)
            
if (FN_KNN+TP_KNN) > 0:
    FNR_KNN=FN_KNN/(FN_KNN+TP_KNN)
    print("The FNR_KNN result:", FNR_KNN)
            
if ((TN_KNN/(TN_KNN+FP_KNN))+(TP_KNN/(TP_KNN+FP_KNN))) > 0:
    AUC_KNN=1/(2*((TN_KNN/(TN_KNN+FP_KNN))+(TP_KNN/(TP_KNN+FP_KNN))))
    print("The AUC_KNN result:", AUC_KNN)
            
if  (TP_KNN+TN_KNN+FP_KNN+FN_KNN) > 0:
    ACC_KNN=(TP_KNN+TN_KNN)/(TP_KNN+TN_KNN+FP_KNN+FN_KNN)
    print("The ACC_KNN result:", ACC_KNN)
            
if  ((TP_KNN+FP_KNN)*(TP_KNN+FN_KNN)*(TN_KNN+FP_KNN)*(TN_KNN+FN_KNN)) > 0:
    MCC_KNN=(TP_KNN*TN_KNN-FP_KNN*FN_KNN)/math.sqrt((TP_KNN+FP_KNN)*(TP_KNN+FN_KNN)*(TN_KNN+FP_KNN)*(TN_KNN+FN_KNN))
    print("The Matthews correlation coefficient result:", MCC_KNN)
print("                                                                                                                               ")
print("************************************************End of KNN Based Semi-Supervised Defense(KSD) part*****************************")

************************Result After KNN_Based Defense************************************************************************
                                                                                                                               
Accuracy for Train set: 96.053863
Loss for Train set: 0.106215
                                                                   
Accuracy After KNN-Based Defense: 72.515714
Loss After KNN-Based Defense: 5.564616
                                                                   
TN_KNN: 356366
FP_KNN: 98098
FN_KNN: 57504
TP_KNN: 54181
                                                                   
The FPR_KNN result: 0.2158542810871708
The TPR_KNN result: 0.4851233379594395
The TNR_KNN result: 0.7841457189128291
The FNR_KNN result: 0.5148766620405605
The AUC_KNN result: 0.4386170429645779
The ACC_KNN result: 0.7251571582745885
The Matthews correlation coefficient result: 4.958982233531698
                                         

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45: RuntimeWarning: overflow encountered in long_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:46: RuntimeWarning: overflow encountered in long_scalars
